<a href="https://colab.research.google.com/github/hasankere/Week5_Challenge_project/blob/main/ethiomart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForTokenClassification, AdamW
from transformers import get_scheduler
import torch
from torch.utils.data import DataLoader, Dataset
from seqeval.metrics import classification_report
import sqlite3


In [70]:
from google.colab import files

# Upload file
uploaded = files.upload()



Saving channels_to_crawls.csv to channels_to_crawls (3).csv


In [71]:
import pandas as pd

# Load the uploaded file
channels_df = pd.read_csv('channels_to_crawls.csv', header=None)

# Use the first column as the channel list
channel_list = channels_df[0].tolist()
print(channel_list)


['@ZemenExpress', '@nevacomputer', '@meneshayeofficial', '@ethio_brand_collection', '@Leyueqa', '@sinayelj', '@Shewabrand', '@helloomarketethiopia', '@modernshoppingcenter', '@qnashcom', '@Fashiontera', '@kuruwear', '@gebeyaadama', '@MerttEka', '@forfreemarket', '@classybrands', '@marakibrand', '@aradabrand2', '@marakisat2', '@belaclassic', '@AwasMart', '@qnashcom', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n

In [72]:
import nest_asyncio
nest_asyncio.apply()  # Allow nested event loops


In [73]:
import time
import asyncio

async def fetch_with_retry(channel_info, limit=100, retries=3):
    attempt = 0
    while attempt < retries:
        try:
            return await fetch_messages(channel_info, limit)
        except OperationalError as e:
            print(f"Database locked, retrying... Attempt {attempt + 1}/{retries}")
            await asyncio.sleep(5)  # Wait 5 seconds before retrying
            attempt += 1
    raise Exception("Max retries reached, database still locked.")


In [74]:
import asyncio
import time
from telethon.errors import SessionPasswordNeededError
from sqlite3 import OperationalError

async def fetch_with_retry(channel_info, limit=100, retries=3):
    attempt = 0
    while attempt < retries:
        try:
            return await fetch_messages(channel_info, limit)  # Replace with your message-fetching function
        except OperationalError as e:
            print(f"Database is locked, retrying... Attempt {attempt + 1}/{retries}")
            await asyncio.sleep(5)  # Wait 5 seconds before retrying
            attempt += 1
    raise Exception("Max retries reached, database still locked.")


In [79]:
from telethon import TelegramClient

# Specify the session filename (it will be saved in the current working directory)
session_filename = 'my_session'

# Create the TelegramClient
client = TelegramClient(session_filename, api_id, api_hash)

async def main():
    await client.start()
    print("Client started")
    # Perform other operations like fetching messages
    await client.disconnect()  # Disconnect after use

# Run the client
import asyncio
asyncio.run(main())




Please enter your phone (or bot token): +251912026082
Please enter the code you received: 89204
Signed in successfully as Hasan Kere Obse; remember to not break the ToS or you will risk an account ban!
Client started


In [84]:
# Use a unique session name for each client
client_1 = TelegramClient('session_name_1', api_id, api_hash)
client_2 = TelegramClient('session_name_2', api_id, api_hash)



In [102]:
import pandas as pd
from telethon import TelegramClient
import nest_asyncio
import asyncio
from google.colab import files
import time

# Allow nested event loops for Google Colab
nest_asyncio.apply()

# Telegram API credentials
api_id = '27904749'
api_hash = '5d9f2efaa785a2c4d04b5043088c2813'

# Load the uploaded channels CSV
channels_df = pd.read_csv('channels_to_crawls.csv', header=None)
channel_list = channels_df[0].tolist()  # Use the first column as the channel list

# Initialize Telegram Client
client = TelegramClient(':memory:', api_id, api_hash)

# Retry logic for handling operational errors
def retry_on_lock(func, retries=3, delay=5):
    for _ in range(retries):
        try:
            return func()
        except Exception as e:
            print(f"Error: {e}, retrying in {delay} seconds...")
            time.sleep(delay)
    raise Exception("Retries exhausted.")

# Function to fetch messages from a single channel
async def fetch_messages(channel_username, limit=100):
    messages = []
    try:
        async for message in client.iter_messages(channel_username, limit=limit):
            media_path = None
            if message.media:
                media_path = await message.download_media(file='media/')

            messages.append({
                'channel_username': channel_username,
                'id': message.id,
                'message': message.text,
                'date': message.date,
                'media_path': media_path
            })
    except Exception as e:
        print(f"Error fetching messages from {channel_username}: {e}")
    return messages

# Fetch messages from all channels
async def fetch_from_all_channels():
    all_data = []
    for channel_username in channel_list:
        print(f"Fetching messages from {channel_username}...")
        channel_data = await fetch_messages(channel_username)
        all_data.extend(channel_data)
    return all_data

# Main coroutine
async def main():
    retry_on_lock(lambda: asyncio.run(client.start()))
    async with client:
        all_data = await fetch_from_all_channels()

    # Save data to CSV
    df = pd.DataFrame(all_data)
    output_path = '/content/raw_telegram_data.csv'
    df.to_csv(output_path, index=False)
    files.download(output_path)

    print(df.head())

# Run the main coroutine
asyncio.run(main())


Fetching messages from @ZemenExpress...
Fetching messages from @nevacomputer...
Fetching messages from @meneshayeofficial...
Fetching messages from @ethio_brand_collection...
Fetching messages from @Leyueqa...
Fetching messages from @sinayelj...
Fetching messages from @Shewabrand...
Fetching messages from @helloomarketethiopia...
Fetching messages from @modernshoppingcenter...
Fetching messages from @qnashcom...
Fetching messages from @Fashiontera...
Fetching messages from @kuruwear...
Fetching messages from @gebeyaadama...
Fetching messages from @MerttEka...
Fetching messages from @forfreemarket...
Fetching messages from @classybrands...
Fetching messages from @marakibrand...
Fetching messages from @aradabrand2...
Fetching messages from @marakisat2...
Fetching messages from @belaclassic...
Fetching messages from @AwasMart...
Fetching messages from @qnashcom...
Fetching messages from nan...
Error fetching messages from nan: Cannot cast float to any kind of Peer.
Fetching messages from 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  channel_username    id                                            message  \
0    @ZemenExpress  6020  💥💥...................................💥💥\n\n📌 R...   
1    @ZemenExpress  6019                                                      
2    @ZemenExpress  6018                                                      
3    @ZemenExpress  6017                                                      
4    @ZemenExpress  6016  💥💥...................................💥💥\n\n📌 2...   

                       date                               media_path  
0 2025-01-16 12:49:25+00:00                               media/.jpg  
1 2025-01-16 06:29:40+00:00      media/photo_2025-01-16_06-29-40.jpg  
2 2025-01-16 06:29:40+00:00  media/photo_2025-01-16_06-29-40 (1).jpg  
3 2025-01-16 06:29:40+00:00  media/photo_2025-01-16_06-29-40 (2).jpg  
4 2025-01-16 06:29:40+00:00  media/photo_2025-01-16_06-29-40 (3).jpg  


In [103]:
import nltk

# Ensure 'punkt' tokenizer is downloaded
nltk.download('punkt')  # This is the standard tokenizer
nltk.download('stopwords')  # Optional: if you need stopwords for filtering

# Import necessary libraries
import pandas as pd
import string

# Function to preprocess Amharic text
def preprocess_amharic_text(text):
    # Convert the text to string if it's not already
    if isinstance(text, float):
        text = str(text)

    # Step 1: Normalize text - Remove diacritics (manual approach)
    normalized_text = ''.join([char for char in text if char not in ['፣', '፤', '፦', '፧', '፩', '፮', '፱', '፴', '፻']])

    # Step 2: Remove punctuation
    text_no_punctuation = ''.join([char for char in normalized_text if char not in string.punctuation])

    # Step 3: Tokenize text using nltk's word_tokenize
    try:
        tokens = nltk.word_tokenize(text_no_punctuation)
    except LookupError as e:
        print("Error during tokenization:", e)
        tokens = []  # If tokenization fails, return an empty list

    # Step 4: Convert to lowercase
    normalized_lower_text = ' '.join([word.lower() for word in tokens])

    return normalized_lower_text

# Load the CSV file with the filtered data (ensure it's uploaded to Colab first)
df = pd.read_csv('/content/telegram_messages.csv')

# Apply the preprocessing to the 'content' column (or the relevant column in your data)
df['processed_content'] = df['content'].apply(preprocess_amharic_text)

# Save the processed data to a new CSV file
output_path = '/content/processed_telegram_messages.csv'
df.to_csv(output_path, index=False)

# Download the file
from google.colab import files
files.download(output_path)

# Display the first few rows of the original and processed content
df[['content', 'processed_content']].head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Streaming output truncated to the last 5000 lines.
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Error during tokenization: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Error 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,content,processed_content
0,NaN,
1,NaN,
2,NaN,
3,💥💥...................................💥💥\n\n📌 2...,
4,💥💥...................................💥💥\n\n📌 2...,


In [ ]:
import pandas as pd
from google.colab import files
import string
import nltk

# Ensure the necessary NLTK data is downloaded
nltk.download('punkt')

# Function to preprocess and clean Amharic text
def preprocess_amharic_text(text):
    """Preprocess text: remove diacritics, punctuation, and tokenize."""
    # Convert the text to string if it's not already
    if isinstance(text, float):
        text = str(text)

    # Step 1: Normalize text - Remove diacritics (manual approach for common characters)
    normalized_text = ''.join([char for char in text if char not in ['፣', '፤', '፦', '፧', '፩', '፮', '፱', '፴', '፻']])

    # Step 2: Remove punctuation
    text_no_punctuation = ''.join([char for char in normalized_text if char not in string.punctuation])

    # Step 3: Tokenize text using nltk's word_tokenize
    try:
        tokens = nltk.word_tokenize(text_no_punctuation)
    except LookupError as e:
        print("Error during tokenization:", e)
        tokens = []  # If tokenization fails, return an empty list

    # Step 4: Convert to lowercase
    normalized_lower_text = ' '.join([word.lower() for word in tokens])

    return normalized_lower_text

# Load the CSV file (ensure the file has been uploaded to Colab)
df = pd.read_csv('/content/telegram_messages.csv')

# Step 4: Clean and structure the data
# We want to separate metadata and message content:
df['processed_content'] = df['content'].apply(preprocess_amharic_text)

# Ensure to separate metadata: sender_id, timestamp, and message content
df_cleaned = df[['sender', 'timestamp', 'processed_content']].copy()

# Step 5: Store preprocessed data in a structured format
# Save the cleaned data to a new CSV file
output_path = '/content/processed_telegram_messages.csv'
df_cleaned.to_csv(output_path, index=False)

# Optionally: Download the file for local use
files.download(output_path)

# Display the first few rows of the cleaned data
df_cleaned.head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Streaming output truncated to the last 5000 lines.
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Error during tokenization: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Error 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,sender,timestamp,processed_content
0,-1001307493052,2025-01-16 06:29:40+00:00,
1,-1001307493052,2025-01-16 06:29:40+00:00,
2,-1001307493052,2025-01-16 06:29:40+00:00,
3,-1001307493052,2025-01-16 06:29:40+00:00,
4,-1001307493052,2025-01-16 06:28:10+00:00,


In [104]:
import pandas as pd

# Sample data, assuming you have a DataFrame from your telegram messages
df = pd.read_csv('/content/processed_telegram_messages.csv')

# Function to convert text into CoNLL format (this is a simple example)
def label_in_conll_format(text):
    """
    This function will label a text using a simple example for NER.
    You should replace this logic with your own manual or automated labeling mechanism.
    """
    # Check if the text is a valid string (not NaN)
    if not isinstance(text, str):
        text = ""  # If not a string, replace with empty string

    # Example predefined labels for tokens
    words = text.split()
    labeled_text = []

    for word in words:
        if word.lower() in ['john', 'doe']:  # Example for a person's name
            labeled_text.append(f"{word} B-PER")  # Label 'John' and 'Doe' as a person
        elif word.lower() in ['addis', 'ababa']:  # Example for a location
            labeled_text.append(f"{word} B-LOC")  # Label 'Addis' and 'Ababa' as a location
        else:
            labeled_text.append(f"{word} O")  # Label others as non-entity

    return "\n".join(labeled_text) + "\n"  # Return formatted text in CoNLL format

# Function to save labeled data into CoNLL format
def save_to_conll(df, output_path):
    with open(output_path, 'w') as f:
        for index, row in df.iterrows():
            # Apply the labeling function to each content column
            labeled_text = label_in_conll_format(row['processed_content'])
            f.write(labeled_text)
            f.write("\n")  # Add an empty line after each sentence

# Call the function to save data in CoNLL format
output_path = '/content/labeled_telegram_data.conll'
save_to_conll(df, output_path)

# Display the file path where the labeled data is saved
print(f"Data has been labeled and saved to: {output_path}")

# Optionally: Download the file for local use
from google.colab import files
files.download(output_path)


Data has been labeled and saved to: /content/labeled_telegram_data.conll


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [105]:
import pandas as pd

# Dummy function to demonstrate how to label entities (replace this with your actual labeling logic)
def label_in_conll_format(text):
    """
    Function to label tokens (words) in the text. Replace this function with the logic for NER labeling.
    The function returns the tokens in CoNLL format with predefined entity labels.
    """
    # Tokenize the text (simple example, you may need a more sophisticated tokenizer)
    tokens = text.split()

    labeled_text = []

    for token in tokens:
        # Example: Assign labels based on certain conditions (replace with your actual logic)

        # English entity recognition
        if token.lower() in ['product', 'item']:  # Example condition for English product
            label = 'PRODUCT'
        elif token.lower() in ['etb', 'usd']:  # Example: label currencies like ETB, USD as PRICE
            label = 'PRICE'
        elif token.lower() in ['addis', 'ethiopia']:  # Example: label cities/countries as LOCATION
            label = 'LOCATION'

        # Amharic entity recognition (example words, modify as needed)
        elif token in ['እቃ', 'አምስት']:  # 'እቃ' is Amharic for 'product', 'አምስት' is 'five'
            label = 'PRODUCT'
        elif token in ['ብር', 'ኢትቢ'] :  # Amharic currencies (like 'ብር' for Birr, 'ኢትቢ' for ETB)
            label = 'PRICE'
        elif token in ['አዲስ', 'ኢትዮጵያ']:  # 'አዲስ' for Addis Ababa and 'ኢትዮጵያ' for Ethiopia
            label = 'LOCATION'
        else:
            label = 'O'  # O stands for "Other", no entity

        labeled_text.append(f"{token} {label}")

    # Return the labeled text in CoNLL format (each token + label on its own line)
    return "\n".join(labeled_text)

def save_to_conll(df, output_path):
    """
    Function to save labeled data in CoNLL format (each token is labeled in a separate line).
    """
    with open(output_path, 'w') as f:
        for index, row in df.iterrows():
            # Ensure that 'processed_content' column is not empty or NaN before labeling
            if isinstance(row['processed_content'], str) and row['processed_content'].strip():
                labeled_text = label_in_conll_format(row['processed_content'])
                f.write(labeled_text)
                f.write("\n")  # Add an empty line after each sentence

# Assuming you have already preprocessed the data and stored it in 'df'
# Here's an example dataframe (including both English and Amharic)
df = pd.DataFrame({
    'sender': [12345, 67890],
    'timestamp': ['2025-01-16 10:00', '2025-01-16 10:05'],
    'processed_content': ['This is a product costing 100 ETB in Addis Ababa.',
                          'እቃ ሲሆን በ አዲስ ኢትዮጵያ 200 ብር ይዘዋል።']  # Example Amharic message
})

# Save the labeled data to a CoNLL file
output_path = '/content/labeled_telegram_data.conll'  # Save to Colab's environment
save_to_conll(df, output_path)

# Display the file path where the labeled data is saved
print(f"Labeled data saved to {output_path}")


Labeled data saved to /content/labeled_telegram_data.conll


In [106]:
import pandas as pd

# Function to label tokens in CoNLL format (with B- and I- for entities)
def label_in_conll_format(text):
    """
    Function to label tokens (words) in the text. Replace this function with the logic for NER labeling.
    The function returns the tokens in CoNLL format with predefined entity labels.
    """
    tokens = text.split()
    labeled_text = []

    # Variables to track entity labeling (for B- and I- tags)
    current_entity = None  # Keeps track of the current entity
    for i, token in enumerate(tokens):
        # Product Entity Labeling
        if token.lower() in ['baby', 'bottle', 'laptop', 'phone', 'chair']:  # Example products
            if current_entity != 'PRODUCT':
                label = 'B-PRODUCT'  # Start of a new product entity
                current_entity = 'PRODUCT'
            else:
                label = 'I-PRODUCT'  # Inside the product entity
        # Location Entity Labeling
        elif token.lower() in ['addis', 'ababa', 'bole', 'ethiopia']:  # Example locations
            if current_entity != 'LOC':
                label = 'B-LOC'  # Start of a new location entity
                current_entity = 'LOC'
            else:
                label = 'I-LOC'  # Inside the location entity
        # Price Entity Labeling
        elif token.isdigit() or token in ['ብር', 'etb']:  # Example for price (ETB, digits)
            if current_entity != 'PRICE':
                label = 'B-PRICE'  # Start of a new price entity
                current_entity = 'PRICE'
            else:
                label = 'I-PRICE'  # Inside the price entity
        else:
            label = 'O'  # Outside any entity (e.g., common words or punctuation)
            current_entity = None  # Reset entity if no match

        labeled_text.append(f"{token} {label}")

    # Return the labeled text in CoNLL format (each token + label on its own line)
    return "\n".join(labeled_text)

def save_to_conll(df, output_path):
    """
    Function to save labeled data in CoNLL format (each token is labeled in a separate line).
    """
    with open(output_path, 'w') as f:
        for index, row in df.iterrows():
            # Ensure that 'processed_content' column is not empty or NaN before labeling
            if isinstance(row['processed_content'], str) and row['processed_content'].strip():
                labeled_text = label_in_conll_format(row['processed_content'])
                f.write(labeled_text)
                f.write("\n\n")  # Add an empty line after each sentence

# Assuming you have already preprocessed the data and stored it in 'df'
# Here's an example dataframe (including both English and Amharic)
df = pd.DataFrame({
    'sender': [12345, 67890],
    'timestamp': ['2025-01-16 10:00', '2025-01-16 10:05'],
    'processed_content': [
        'This is a baby bottle costing 100 ETB in Addis Ababa.',
        'እቃ ሲሆን በ አዲስ ኢትዮጵያ 200 ብር ይዘዋል።'
    ]  # Example messages in both English and Amharic
})

# Save the labeled data to a CoNLL file
output_path = '/content/labeled_telegram_data.conll'  # Save to Colab's environment
save_to_conll(df, output_path)

# Display the file path where the labeled data is saved
print(f"Labeled data saved to {output_path}")


Labeled data saved to /content/labeled_telegram_data.conll
